In [1]:
import torch 
from torch.utils.data import DataLoader
from torch.nn import MSELoss
import numpy as np
import pickle
import math
import time

from sys_id.dataset import load_trajectory, WheeledTrajWindowed, PhysProps
from model import GPT2


In [2]:
eval_params = {
        'checkpoint_path': './logs/2024-04-07_14-55-49/checkpoint_epoch_8500.pth', 
        'dataset_folder_path': '../dataset/eval_model/wheeled_flat', 
        'window_size': 10,
        'batch_size': 1, 
    }

model_params = {
        "n_layer": 2,
        "n_head": 3,
        "pdrop": 0.1,
        "max_seq_length": 1000,
        'position_encoding': 'sine',
        "output_size": 3,
        "input_size": (42 + 12) * eval_params['window_size'], 
        "hidden_size": (42 + 12) * eval_params['window_size'], 
    }

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPT2(**model_params).to(device)
optimizer = torch.optim.Adam(model.parameters())
checkpoint = torch.load(eval_params['checkpoint_path'], map_location=device)
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [4]:
import pickle as pkl 
import os
file_path = os.path.join(eval_params["dataset_folder_path"], "traj_0000.pkl")
with open(file_path, "rb") as file:
    traj = pkl.load(file)

print(traj.keys())

dict_keys(['obs', 'act', 'physprops'])


In [18]:
obs_history = np.array(traj['obs'][:10]).flatten()
action_history = np.array(traj['act'][:10]).flatten()
tmp_history = np.concatenate([obs_history, action_history])
history_input = torch.tensor(tmp_history, device = device)
print(history_input.shape)

torch.Size([540])


In [19]:
model.eval()
estimation, _ = model(history_input, None)

RuntimeError: expected scalar type Double but found Float